In [1]:
%pip install --upgrade langchain langchain-ollama langchain faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.4
    Uninstalling langchain-1.2.4:
      Successfully uninstalled langchain-1.2.4


In [9]:
%pip install -qU  langchain-community "docarray"

In [2]:
import os
import requests
from PIL import Image

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, Markdown

# Installing and Running Ollama Locally

### Step 1: Download and install Ollama

In [4]:
!sudo apt-get install zstd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 0s (2,032 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package zstd.
(Reading database ... 117540 files and directories currently 

In [5]:
# Download the Ollama Linux binary
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


### Step 2: Start Ollama server in the background

In [59]:
import subprocess
import os

# Set the path to the Ollama binary
ollama_path = "/usr/local/bin/ollama"

# Start Ollama server in the background
process = subprocess.Popen([ollama_path, "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print("Ollama server started in the background.")

# Optional: Give it a moment to fully start
import time
time.sleep(5)
print("Waiting for Ollama server to initialize...")

Ollama server started in the background.
Waiting for Ollama server to initialize...


###Step 3: Pull the required model

*(Note: This step might take a while depending on the model size and your internet connection.)*

In [7]:
!ollama pull llama3.2-vision
print("Model llama3.2-vision pulled.")


Model llama3.2-vision pulled.


In [10]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_ollama import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader

In [11]:
%pip install -U langchain-text-splitters
%pip install -qU faiss-cpu

In [12]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [60]:
llm = ChatOllama(
    model="llama3.2-vision",
    temperature=1.0,
    max_tokens=400,
    timeout=None,
    max_retries=3
)

In [14]:
import requests
from PIL import Image

def get_image(url, path):
  response = requests.get(url).content
  with open(path, "wb") as file:
    file.write(response)
  image = Image.open(path)
  return image

In [21]:
def load_text(path):
  loader = TextLoader(path)
  text = loader.load()
  return text[0].page_content

In [22]:
text = load_text("/content/data/state_of_the_union.txt")

In [25]:
from langchain_text_splitters import CharacterTextSplitter

def get_chunks(text):
  text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=10)
  chunks = text_splitter.split_text(text)
  docs = [Document(page_content=x) for x in chunks]
  return docs

In [26]:
docs = get_chunks(text)

In [46]:
from langchain_ollama import OllamaEmbeddings

embedding_model = OllamaEmbeddings(
    model="nomic-embed-text"
)

In [50]:
vectorstore = FAISS.from_documents(docs, embedding=embedding_model)

In [51]:
retriever = vectorstore.as_retriever()

In [31]:
retriever.invoke("What is USA's response to what had happened with Ukraine?")

[Document(id='4c77ed7d-89ac-4fc9-a93d-bd724452fa97', metadata={}, page_content='There’s something happening in America.'),
 Document(id='9b424699-65ac-4c12-b8fe-cc64e1f4001c', metadata={}, page_content='Headaches. Numbness. Dizziness.'),
 Document(id='4703d6c6-dd8d-4d41-bf88-4273a1f281ec', metadata={}, page_content='Putin’s latest attack on Ukraine was premeditated and unprovoked.'),
 Document(id='20bf10ea-4fa5-474e-83c9-196a8520c7dc', metadata={}, page_content='We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever.')]

In [32]:
prompt_template = """
```
{context}
```

{query}

Provide brief information and future aspects.
"""

In [33]:
prompt = ChatPromptTemplate.from_template(prompt_template)

In [34]:
from langchain_core.runnables import RunnablePassthrough

In [61]:
chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [36]:
from langchain_core.messages import HumanMessage, SystemMessage

In [39]:
image = get_image("https://ichef.bbci.co.uk/news/1024/cpsprodpb/d510/live/60d85bb0-f5f5-11f0-b5f7-49f0357294ff.jpg.webp", path="/content/image.jpg")

In [64]:
llm_response = chain.invoke("Discuss in very brief how US is helping Ukraine?")

In [65]:
display(Markdown(llm_response))

The US is providing over $1 billion in direct assistance to Ukraine. Additionally, the US is "inflicting pain on Russia" by supporting Ukraine and attempting to isolate Putin from the world. The US will continue to aid the Ukrainian people as they defend their country and alleviate their suffering.

Below code will not work with the existing rag chain as the embedding model we are using only supports text not images.

In [41]:
messages = [
    SystemMessage(content="You are a helpful assistant which takes both images and text prompts and answers in breif."),
    HumanMessage(content=[
        {"type": "text", "text": "Discuss USA's response to Russian Invasion of Ukraine."},
        {"type": "image_url", "image_url": image}
    ])
]

In [51]:
llm_response = chain.invoke(messages)